<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# MTBS monthly downsampling and reprojection

_by Joe Hamman (CarbonPlan), August 5, 2020_

This notebook downsamples and reprojects monthly MTBS 30m rasters stored in
Cloud Optimized GeoTIFF 4000m GeoTIFFs.

**Inputs:**

- Monthly COGs

**Outputs:**

- COG outputs after downsampling and reprojection

**Notes:**

- Source CRS and projection extent come from MTBS


In [ ]:
import os

In [ ]:
from carbonplan_data.utils import projections, setup

workdir, upload = setup("jeremy")
workdir

In [ ]:
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

dst_profile = cog_profiles.get("deflate")

In [ ]:
def get_file(region, fire, year, month):
    return {
        "source": (
            workdir
            / f"processed/mtbs/{region}/30m/{fire}_{year}.{month:02g}.tif"
        ).as_posix(),
        "target": f"processed/mtbs/{region}/4000m/tif/{fire}.{month:02g}.tif",
    }

In [ ]:
resolution = 4000

for year in [1984]:
    for month in [1]:
        for fire in ["vlf"]:
            for region in ["ak", "conus"]:
                f = get_file(region, fire, year, month)
                crs, extent = projections("albers", region)
                resampling = "average"
                cmd = (
                    "gdalwarp "
                    "-t_srs '%s' "
                    "-te %s "
                    "-tr %s %s "
                    "-r %s "
                    "%s "
                    "%s"
                ) % (
                    crs,
                    extent,
                    resolution,
                    resolution,
                    resampling,
                    f["source"],
                    "./raster.tif",
                )
                os.system(cmd)
                cog_translate("./raster.tif", "./raster.tif", dst_profile)
                upload("./raster.tif", f["target"])
                os.remove("./raster.tif")

In [ ]:
files = [get_file("conus", "vlf", 1984, month)["source"] for month in range(12)]

In [ ]:
resolution = 4000

for year in [2017]:
    for region in ["conus"]:

        for fire in ["vlf"]:
            files = [
                get_file("conus", "vlf", year, month + 1)["source"]
                for month in range(12)
            ]
            crs, extent = projections("albers", region)
            resampling = "sum"
            cmd = (
                "gdalwarp "
                "-t_srs '%s' "
                "-te %s "
                "-tr %s %s "
                "-r %s "
                "%s "
                "%s"
            ) % (
                crs,
                extent,
                resolution,
                resolution,
                resampling,
                " ".join(files),
                "./raster.tif",
            )
            print(cmd)
            os.system(cmd)
            # cog_translate("./raster.tif", "./raster.tif", dst_profile)
            # upload("./raster.tif", f["target"])
            # os.remove("./raster.tif")

In [ ]:
import rasterio

r = rasterio.open(
    "/Users/freeman/workdir/carbonplan-data/processed/mtbs/conus/30m/vlf_2018.09.tif"
)
im = r.read(1)

In [ ]:
r = rasterio.open("./raster.tif")
im = r.read(1)

In [ ]:
%matplotlib inline

In [ ]:
from showit import image

In [ ]:
im.sum()

In [ ]:
image(im, size=12, clim=(0, 1))

In [ ]:
from carbonplan_forests import load

In [ ]:
mtbs = load.mtbs(store="local")

In [ ]:
before = mtbs.groupby("time.year").sum().sel(year=2018)["vlf"]

In [ ]:
image(1 - before, size=12, clim=(0, 1))